In [29]:
import pandas as pd

In [107]:
train = pd.read_csv('Downloads/trainMSCI.csv')

In [152]:
test = pd.read_csv('Downloads/testMSCI2.csv')

In [109]:
test

,Headline,Body ID,Stance,articleBody,isRelated
0,ferguson riots pregnant woman loses eye cops f...,2008,unrelated,respected senior french police officer investi...,0
1,crazy conservatives sure gitmo detainee killed...,1550,unrelated,dave morins social networking company path rep...,0
2,russian guy says justin bieber ringtone saved ...,2,unrelated,bereaved afghan mother took revenge taliban wa...,0
3,zombie cat buried kitty believed dead meows ba...,1793,unrelated,hewlettpackard officially splitting two follow...,0
4,argentinas president adopts boy end werewolf c...,37,unrelated,airline passenger headed dallas removed plane ...,0
...,...,...,...,...,...
25408,success affordable care act hugely inconvenien...,2582,agree,congressional republicans evidently hoping rep...,1
25409,success affordable care act hugely inconvenien...,2583,discuss,obamacare work worth reflecting upon president...,1
25410,success affordable care act hugely inconvenien...,2584,disagree,millions may lose coverage next year congress ...,1
25411,success affordable care act hugely inconvenien...,2585,disagree,come november grim trudge across increasingly ...,1


In [110]:
import keras
from keras.preprocessing.text import Tokenizer

# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 3500
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 100
# This is fixed.
EMBEDDING_DIM = 100
tokenizer1 = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer1.fit_on_texts(train['Headline'].values)
word_index = tokenizer1.word_index
print('Found %s unique tokens.' % len(word_index))

Found 3307 unique tokens.


In [111]:
from keras.preprocessing.sequence import pad_sequences
X_train_headlines = tokenizer1.texts_to_sequences(train['Headline'].values)
X_train_headlines = pad_sequences(X_train_headlines, maxlen=MAX_SEQUENCE_LENGTH, padding = 'post')
print('Shape of data tensor:', X_train_headlines.shape)

Shape of data tensor: (49972, 100)


In [112]:
X_test_headlines = tokenizer1.texts_to_sequences(test['Headline'].values)
X_test_headlines = pad_sequences(X_test_headlines, maxlen=MAX_SEQUENCE_LENGTH, padding = 'post')
print('Shape of data tensor:', X_test_headlines.shape)

Shape of data tensor: (25413, 100)


In [113]:

tokenizer2 = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer2.fit_on_texts(train['articleBody'].values)
word_index = tokenizer2.word_index
print('Found %s unique tokens.' % len(word_index))

Found 24783 unique tokens.


In [114]:
X_train_body = tokenizer2.texts_to_sequences(train['articleBody'].values)
X_train_body = pad_sequences(X_train_body, maxlen=MAX_SEQUENCE_LENGTH, padding = 'post')
print('Shape of data tensor:', X_train_body.shape)

Shape of data tensor: (49972, 100)


In [115]:
X_test_body = tokenizer2.texts_to_sequences(test['articleBody'].values)
X_test_body = pad_sequences(X_test_body, maxlen=MAX_SEQUENCE_LENGTH, padding = 'post')
print('Shape of data tensor:', X_test_body.shape)

Shape of data tensor: (25413, 100)


In [116]:
Y_train = pd.get_dummies( train['isRelated']).values
print('Shape of label tensor:', Y_train.shape)

Shape of label tensor: (49972, 2)


In [117]:
Y_test = pd.get_dummies( test['isRelated']).values
print('Shape of label tensor:', Y_test.shape)

Shape of label tensor: (25413, 2)


In [25]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import LSTM, Dense, InputLayer, Concatenate, Input
from keras.models import Sequential
from tensorflow.keras import layers, Model
from keras.layers import Embedding

In [68]:
column1_input = Input(X_train_headlines.shape[1])
column2_input = Input(X_train_body.shape[1])

# column1_LSTM
c0 = Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_train_headlines.shape[1])(column1_input)
c1 = LSTM(64, return_sequences=False)(c0)
c2 = Dense(64)(c1)

# column2_LSTM
d0 = (Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_train_body.shape[1]))(column2_input)
d1 = LSTM(64, return_sequences=False)(d0)
d2 = Dense(64)(d1)

# shared
concat_layer= Concatenate()([c2, d2])
shared1 = Dense(128, activation='relu')(concat_layer)
output = Dense(2, activation='sigmoid')(shared1)

# define a model with a list of two inputs
model = Model(inputs=[column1_input, column2_input], outputs=output)
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [70]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Total params: 709,570
Trainable params: 709,570
Non-trainable params: 0
__________________________________________________________________________________________________


In [43]:
epochs = 5
batch_size = 64

history = model.fit([X_train_headlines,X_train_body ], Y_train, epochs=epochs, batch_size=batch_size)

Epoch 1/5
781/781 [==============================] - 81s 100ms/step - loss: 0.4856
Epoch 2/5
781/781 [==============================] - 77s 99ms/step - loss: 0.4144
Epoch 3/5
781/781 [==============================] - 78s 100ms/step - loss: 0.3929
Epoch 4/5
781/781 [==============================] - 80s 103ms/step - loss: 0.3853
Epoch 5/5
781/781 [==============================] - 88s 112ms/step - loss: 0.3808


In [45]:
preds = model.predict([X_test_headlines,X_test_body ])


In [46]:
preds

array([[0.32616723, 0.6698896 ],
       [0.526286  , 0.4759713 ],
       [0.7264933 , 0.2725665 ],
       ...,
       [0.62370485, 0.37320465],
       [0.69233096, 0.3082329 ],
       [0.7433839 , 0.25685298]], dtype=float32)

In [50]:
import numpy as np
preds = np.argmax(preds, axis =1)

In [49]:
y_true = test['isRelated']

In [52]:
from sklearn.metrics import confusion_matrix,f1_score,classification_report


f1 = f1_score(y_true,preds)
print('F1 score',f1)
print('Classification Report')
print(classification_report(y_true,preds))


F1 score 0.23841241502437682
Classification Report
              precision    recall  f1-score   support

           0       0.70      0.69      0.69     18349
           1       0.23      0.25      0.24      7064

    accuracy                           0.56     25413
   macro avg       0.47      0.47      0.47     25413
weighted avg       0.57      0.56      0.57     25413



In [53]:
#Retraining with class weights

In [55]:

class_weight = {0: 1.,
                1: 2.}
history = model.fit([X_train_headlines,X_train_body ], Y_train,class_weight =class_weight, epochs=epochs, batch_size=batch_size)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/5
781/781 [==============================] - 81s 100ms/step - loss: 0.7063 - accuracy: 0.7644
Epoch 2/5
781/781 [==============================] - 73s 94ms/step - loss: 0.6122 - accuracy: 0.8050
Epoch 3/5
781/781 [==============================] - 71s 91ms/step - loss: 0.5914 - accuracy: 0.8100
Epoch 4/5
781/781 [==============================] - 75s 96ms/step - loss: 0.5806 - accuracy: 0.8142
Epoch 5/5
781/781 [==============================] - 78s 99ms/step - loss: 0.5757 - accuracy: 0.8178


In [56]:
preds = model.predict([X_test_headlines,X_test_body ])
preds = np.argmax(preds, axis =1)

In [57]:
f1 = f1_score(y_true,preds)
print('F1 score',f1)
print('Classification Report')
print(classification_report(y_true,preds))


F1 score 0.34606133493686114
Classification Report
              precision    recall  f1-score   support

           0       0.71      0.49      0.58     18349
           1       0.27      0.49      0.35      7064

    accuracy                           0.49     25413
   macro avg       0.49      0.49      0.46     25413
weighted avg       0.59      0.49      0.51     25413



In [58]:
#we see improvement in predicting class 1, and therefore as class 1 has higher weightage, using class weights is good approach

In [140]:
#Next we try to use only one lstm model to predict all 4 classes rather than the two stage appraoch
from keras.layers import SpatialDropout1D


In [141]:
column1_input = Input(X_train_headlines.shape[1])
column2_input = Input(X_train_body.shape[1])

# column1_LSTM
c0 = Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_train_headlines.shape[1])(column1_input)
c1 = SpatialDropout1D(0.2)(c0)
c2 = LSTM(128, return_sequences=False, dropout=0.2, recurrent_dropout=0.2)(c1)
c3 = Dense(64)(c2)

# column2_LSTM
d0 = (Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_train_body.shape[1]))(column2_input)
d1 = SpatialDropout1D(0.2)(d0)
d2 = LSTM(128, return_sequences=False, dropout=0.2, recurrent_dropout=0.2)(d1)
d3 = Dense(64)(d2)

# shared
concat_layer= Concatenate()([c3, d3])
shared1 = Dense(64, activation='relu')(concat_layer)
output = Dense(4, activation='softmax')(shared1)

# define a model with a list of two inputs
model = Model(inputs=[column1_input, column2_input], outputs=output)
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [119]:
Y_train = pd.get_dummies( train['Stance']).values
print('Shape of label tensor:', Y_train.shape)

Shape of label tensor: (49972, 4)


In [121]:
Y_test = pd.get_dummies( test['Stance']).values
print('Shape of label tensor:', Y_test.shape)

Shape of label tensor: (25413, 4)


In [142]:
test['Stance'].value_counts()

unrelated    18349
discuss       4464
agree         1903
disagree       697
Name: Stance, dtype: int64

In [144]:
class_weight = {0: 2.5,
                1: 3,
                2: 2,
                3: 1}
history = model.fit([X_train_headlines,X_train_body ], Y_train,class_weight =class_weight, epochs=epochs, batch_size=batch_size)

Epoch 1/5
781/781 [==============================] - 229s 288ms/step - loss: 1.1765 - accuracy: 0.7502
Epoch 2/5
781/781 [==============================] - 276s 354ms/step - loss: 0.9732 - accuracy: 0.7738
Epoch 3/5
781/781 [==============================] - 298s 381ms/step - loss: 0.9111 - accuracy: 0.7845
Epoch 4/5
781/781 [==============================] - 300s 384ms/step - loss: 0.8778 - accuracy: 0.7904
Epoch 5/5
781/781 [==============================] - 303s 388ms/step - loss: 0.8617 - accuracy: 0.7911


In [146]:
preds = model.predict([X_test_headlines,X_test_body ])
preds = np.argmax(preds, axis =1)

In [ ]:
#unrelated = 3
#agree = 0
#discuss =2
#disagree = 1

In [136]:
test['true'] = 1

In [137]:
test.loc[test.Stance == 'unrelated', 'true'] = 3
test.loc[test.Stance == 'discuss', 'true'] = 2
test.loc[test.Stance == 'agree', 'true'] = 0
test.loc[test.Stance == 'disagree', 'true'] = 1

In [138]:
y_true = test['true']

In [147]:
print('Classification Report')
print(classification_report(y_true,preds))

Classification Report
              precision    recall  f1-score   support

           0       0.09      0.19      0.12      1903
           1       0.00      0.00      0.00       697
           2       0.19      0.31      0.24      4464
           3       0.72      0.55      0.62     18349

    accuracy                           0.47     25413
   macro avg       0.25      0.26      0.25     25413
weighted avg       0.56      0.47      0.50     25413



In [156]:
test['pred'] = preds

In [157]:
test.loc[test.pred == 3, 'Stance'] = 'unrelated'
test.loc[test.pred == 2, 'Stance'] = 'discuss'
test.loc[test.pred == 0, 'Stance'] = 'agree'
test.loc[test.pred == 1, 'Stance'] = 'disagree'

In [159]:
test[['Headline','Body ID', 'Stance']].to_csv('Desktop/answer.csv',index=False, encoding='utf-8')

In [161]:
#Based on results we think 2 stage approach is better and below we continue with the two stage appraoch and build the
#second model

#This is because of data unblancing discussed more in report

In [59]:
#Model 2 predict related type

In [61]:
train2 = train.loc[train['isRelated'] ==1]
test2 = test.loc[test['isRelated'] ==1]

In [62]:
tokenizer3 = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer3.fit_on_texts(train2['Headline'].values)
word_index = tokenizer3.word_index
print('Found %s unique tokens.' % len(word_index))

Found 3307 unique tokens.


In [63]:
X_train_headlines = tokenizer3.texts_to_sequences(train2['Headline'].values)
X_train_headlines = pad_sequences(X_train_headlines, maxlen=MAX_SEQUENCE_LENGTH, padding = 'post')
print('Shape of data tensor:', X_train_headlines.shape)

X_test_headlines = tokenizer3.texts_to_sequences(test2['Headline'].values)
X_test_headlines = pad_sequences(X_test_headlines, maxlen=MAX_SEQUENCE_LENGTH, padding = 'post')
print('Shape of data tensor:', X_test_headlines.shape)

Shape of data tensor: (13427, 100)
Shape of data tensor: (7064, 100)


In [64]:
tokenizer4 = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer4.fit_on_texts(train2['articleBody'].values)
word_index = tokenizer4.word_index
print('Found %s unique tokens.' % len(word_index))

Found 24783 unique tokens.


In [65]:
X_train_body = tokenizer4.texts_to_sequences(train2['articleBody'].values)
X_train_body = pad_sequences(X_train_body, maxlen=MAX_SEQUENCE_LENGTH, padding = 'post')
print('Shape of data tensor:', X_train_body.shape)
X_test_body = tokenizer4.texts_to_sequences(test2['articleBody'].values)
X_test_body = pad_sequences(X_test_body, maxlen=MAX_SEQUENCE_LENGTH, padding = 'post')
print('Shape of data tensor:', X_test_body.shape)

Shape of data tensor: (13427, 100)
Shape of data tensor: (7064, 100)


In [66]:
Y_train = pd.get_dummies( train2['Stance']).values
print('Shape of label tensor:', Y_train.shape)

Y_test = pd.get_dummies( test2['Stance']).values
print('Shape of label tensor:', Y_test.shape)

Shape of label tensor: (13427, 3)
Shape of label tensor: (7064, 3)


In [74]:
column1_input = Input(X_train_headlines.shape[1])
column2_input = Input(X_train_body.shape[1])

# column1_LSTM
c0 = Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_train_headlines.shape[1])(column1_input)
c1 = LSTM(64, return_sequences=False)(c0)
c2 = Dense(64)(c1)

# column2_LSTM
d0 = (Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_train_body.shape[1]))(column2_input)
d1 = LSTM(64, return_sequences=False)(d0)
d2 = Dense(64)(d1)

# shared
concat_layer= Concatenate()([c2, d2])
shared1 = Dense(128, activation='relu')(concat_layer)
output = Dense(3, activation='softmax')(shared1)

# define a model with a list of two inputs
model2 = Model(inputs=[column1_input, column2_input], outputs=output)
model2.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [75]:
history = model2.fit([X_train_headlines,X_train_body ], Y_train, epochs=epochs, batch_size=batch_size)

Epoch 1/5
210/210 [==============================] - 23s 96ms/step - loss: 0.6428 - accuracy: 0.7258
Epoch 2/5
210/210 [==============================] - 19s 89ms/step - loss: 0.4619 - accuracy: 0.8090
Epoch 3/5
210/210 [==============================] - 19s 89ms/step - loss: 0.4084 - accuracy: 0.8256
Epoch 4/5
210/210 [==============================] - 20s 95ms/step - loss: 0.3980 - accuracy: 0.8311
Epoch 5/5
210/210 [==============================] - 22s 104ms/step - loss: 0.3978 - accuracy: 0.8258


In [91]:
preds = model2.predict([X_test_headlines,X_test_body ])
preds = np.argmax(preds, axis =1)

In [84]:
preds

array([2, 2, 0, ..., 2, 2, 2])

In [90]:
#agree = 0
#discuss =2
#disagree 1

test2

,Headline,Body ID,Stance,articleBody,isRelated,true
10,exclusive apple unveil longawaited retina macb...,1964,agree,last week apple sent invites spring forward ev...,1,1
12,found missing afghan soldiers spotted trying e...,2312,discuss,three afghanistan national army officers disap...,2,2
14,report three missing afghan soldiers caught ca...,1754,discuss,toronto three missing afghan soldiers taken cu...,2,2
15,boobed woman fake,1618,agree,woman claiming third breast play hoax us snope...,1,1
20,isis might harvesting organs iraqi ambassador ...,930,discuss,isis using blood money harvesting organs fund ...,2,2
...,...,...,...,...,...,...
25408,success affordable care act hugely inconvenien...,2582,agree,congressional republicans evidently hoping rep...,1,1
25409,success affordable care act hugely inconvenien...,2583,discuss,obamacare work worth reflecting upon president...,2,2
25410,success affordable care act hugely inconvenien...,2584,disagree,millions may lose coverage next year congress ...,1,0
25411,success affordable care act hugely inconvenien...,2585,disagree,come november grim trudge across increasingly ...,1,0


In [86]:
test2['true'] = 0

In [95]:
test2['Stance'].value_counts()

discuss     4464
agree       1903
disagree     697
Name: Stance, dtype: int64

In [89]:
test2.loc[test2.Stance == 'agree', 'true'] = 0
test2.loc[test2.Stance == 'discuss', 'true'] = 2
test2.loc[test2.Stance == 'agree', 'true'] = 1


In [94]:
print('Classification Report')
print(classification_report(test2['true'],preds))


Classification Report
              precision    recall  f1-score   support

           0       0.13      0.53      0.20       697
           1       0.36      0.00      0.00      1903
           2       0.74      0.69      0.71      4464

    accuracy                           0.49      7064
   macro avg       0.41      0.41      0.31      7064
weighted avg       0.58      0.49      0.47      7064



In [96]:
#with class weights

In [98]:
class_weight = {0: 1.7,
                1: 2.7,
               2: 1.}

history = model2.fit([X_train_headlines,X_train_body ], Y_train,class_weight=class_weight, epochs=epochs, batch_size=batch_size)

Epoch 1/5
210/210 [==============================] - 24s 107ms/step - loss: 0.6174 - accuracy: 0.8197
Epoch 2/5
210/210 [==============================] - 21s 99ms/step - loss: 0.5700 - accuracy: 0.8252
Epoch 3/5
210/210 [==============================] - 20s 94ms/step - loss: 0.5730 - accuracy: 0.8246
Epoch 4/5
210/210 [==============================] - 19s 91ms/step - loss: 0.5455 - accuracy: 0.8236
Epoch 5/5
210/210 [==============================] - 20s 94ms/step - loss: 0.5508 - accuracy: 0.8216


In [99]:
preds = model2.predict([X_test_headlines,X_test_body ])
preds = np.argmax(preds, axis =1)

In [100]:
print('Classification Report')
print(classification_report(test2['true'],preds))

Classification Report
              precision    recall  f1-score   support

           0       0.13      0.67      0.22       697
           1       0.28      0.07      0.12      1903
           2       0.78      0.52      0.63      4464

    accuracy                           0.42      7064
   macro avg       0.40      0.42      0.32      7064
weighted avg       0.58      0.42      0.45      7064

